In [1]:
# =====================================
# Imports and Environment Setup
# =====================================
from pycoingecko import CoinGeckoAPI
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
from datetime import datetime
import urllib

In [2]:
# Load environment variables
load_dotenv(override=True)
DB_URL = os.getenv("DB_URL")  # optional if you want to use it

# =====================================
# SQL Server Connection
# =====================================
params = urllib.parse.quote_plus(
    "Driver={ODBC Driver 18 for SQL Server};"
    "Server=OLUWATOSIN,1433;"          # Named instance on fixed port
    "Database=MarketMVP;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")


In [3]:
# Test connection
with engine.connect() as conn:
    print("✅ SQLAlchemy connected!")

✅ SQLAlchemy connected!


In [7]:
# =====================================
# Fetch Crypto Data
# =====================================
cg = CoinGeckoAPI()

def fetch_crypto_data(symbol: str, days: int = 5):
    """
    Fetch crypto data from CoinGecko and aggregate to daily OHLCV.
    """
    data = cg.get_coin_market_chart_by_id(id=symbol, vs_currency='usd', days=days)

    # Create DataFrame with timestamps
    df = pd.DataFrame({
        'timestamp': [datetime.utcfromtimestamp(item[0] // 1000) for item in data['prices']],
        'price': [item[1] for item in data['prices']],
        'market_cap': [item[1] for item in data['market_caps']],
        'volume': [item[1] for item in data['total_volumes']],
        'symbol': symbol
    })

    # Convert timestamp to date
    df['date'] = df['timestamp'].dt.date

    # Aggregate to daily OHLCV
    df_daily = df.groupby('date').agg(
        open_price=('price', 'first'),
        high=('price', 'max'),
        low=('price', 'min'),
        close_price=('price', 'last'),
        volume=('volume', 'sum'),
        market_cap=('market_cap', 'last'),
        symbol=('symbol', 'first')
    ).reset_index()

    return df_daily

In [12]:
# =====================================
# Save to SQL Server (your existing function)
# =====================================
def save_to_db(df: pd.DataFrame, table_name: str, engine=engine):
    """
    Save a DataFrame to SQL Server with proper column name handling.
    """
    # Flatten MultiIndex or tuple columns if any
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join(filter(None, map(str, col))) for col in df.columns]
    else:
        df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

    # Ensure SQL-safe column names
    df.columns = [col.lower().replace(" ", "_").replace("-", "_") for col in df.columns]

    # Insert into SQL Server
    df.to_sql(table_name, engine, if_exists='append', index=False)
    print(f"✅ Data saved to table '{table_name}' successfully!")

In [15]:
# Test run
# =======================
if __name__ == "__main__":
    df = fetch_crypto_data("bitcoin", days=5)
    save_to_db(df, "crypto_data")

✅ Data saved to table 'crypto_data' successfully!


In [16]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

load_dotenv()
engine = create_engine(os.getenv("DB_URL"))

stocks = pd.read_sql("SELECT * FROM stock_data", engine)
cryptos = pd.read_sql("SELECT * FROM crypto_data", engine)

print("Stocks sample:\n", stocks.head())
print("Cryptos sample:\n", cryptos.head())

# Optional: check duplicates
assert not stocks.duplicated().any(), "Duplicate stock rows!"
assert not cryptos.duplicated().any(), "Duplicate crypto rows!"


Stocks sample:
    id ticker        date  open_price    high     low  close_price    volume
0   1   AAPL  2025-09-02      229.25  230.85  226.97       229.72  44075600
1   2   AAPL  2025-09-03      237.21  238.85  234.36       238.47  66427800
2   3   AAPL  2025-09-04      238.45  239.90  236.74       239.78  47549400
3   4   AAPL  2025-09-05      240.00  241.32  238.49       239.69  54870400
4   5   AAPL  2025-09-08      239.30  240.15  236.34       237.88  48999500
Cryptos sample:
          date     open_price           high            low    close_price  \
0  2025-09-11  114136.796349  115251.545046  114136.796349  115251.545046   
1  2025-09-12  115503.166632  116704.714866  114944.793340  116094.986343   
2  2025-09-13  116160.143446  116299.905741  115342.814107  115914.676974   
3  2025-09-14  115974.887394  116143.869783  115264.810470  116031.270652   
4  2025-09-15  115278.552516  116552.241074  114696.006061  115299.780302   

         volume    market_cap   symbol  
0  3.51